In [ ]:
!pip install transformers datasets


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2).to(torch_device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    'csv', 
    data_files={'train': 'en_new_train_split_pegasus_only.csv',
                'test': 'en_new_test_split.csv'}
)

Using custom data configuration default-52efac15bd84d632
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-52efac15bd84d632/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/20 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-52efac15bd84d632/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a/cache-9aa89d8088d7d90b.arrow


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'sentence', 'token_type_ids'],
        num_rows: 19484
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'sentence', 'token_type_ids'],
        num_rows: 3101
    })
})

In [ ]:
# tokenized_dataset.remove_columns(["id"])
tokenized_dataset = tokenized_dataset.rename_column("labels", "label")
tokenized_dataset = tokenized_dataset.rename_column("sentence", "text")

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 19484
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 3101
    })
})

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

model_path = "output/checkpoint-1500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

args = TrainingArguments(
    output_dir="output-pegasus",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset['train'], 
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(tokenized_dataset['test']['sentence'])
# y_pred = np.argmax(raw_pred, axis=1)

loading configuration file output/checkpoint-1500/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file output/checkpoint-1500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceCla

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.129100,1.815809,0.742341,0.725330,0.891061,0.799699
1000,0.091100,2.399658,0.594647,0.630191,0.720670,0.672400
1500,0.074900,2.588794,0.622380,0.747007,0.522905,0.615182
2000,0.066900,2.820958,0.684940,0.671736,0.888268,0.764975


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 3101
  Batch size = 8
Saving model checkpoint to output-pegasus/checkpoint-500
Configuration saved in output-pegasus/checkpoint-500/config.json
Model weights saved in output-pegasus/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 3101
  Batch size = 8
Saving model checkpoint to output-pegasus/checkpoint-1000
Configuration saved in output-pegasus/checkpoint-1000/config.json
Model weights saved in output-pegasus/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.


TrainOutput(global_step=2000, training_loss=0.0905114803314209, metrics={'train_runtime': 3632.3166, 'train_samples_per_second': 16.092, 'train_steps_per_second': 2.012, 'total_flos': 4209776885760000.0, 'train_loss': 0.0905114803314209, 'epoch': 0.82})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text.
***** Running Evaluation *****
  Num examples = 3101
  Batch size = 8


{'epoch': 0.82,
 'eval_accuracy': 0.7423411802644309,
 'eval_f1': 0.7996991727249938,
 'eval_loss': 1.8158094882965088,
 'eval_precision': 0.7253296953160527,
 'eval_recall': 0.8910614525139665,
 'eval_runtime': 200.1259,
 'eval_samples_per_second': 15.495,
 'eval_steps_per_second': 1.939}